The landsat_hires netcdf file can be found at http://glasslab.engr.ccny.cuny.edu/u/hannah/eas213/landsat_hires.nc
The NYC census tracts can be found at https://github.com/story645/EAS213/tree/master/data

If you have the file on your computer, set the path to the location on disk:

```python
datapath = "/path/to/landsat_hires.nc"
shapepath = "/path/to/census.geojson"
```
if you are running the file via mybinder (the interactive version), then set the path to the managed version of the data:

```python
from quilt.data.story645 import eas213
datapath = eas213.landsat_hires._data()
geopath = eas213.census._data()
```

You can also install quilt and the data package on your computer to access the data. To do so, enter the following commands in a terminal or anaconda command prompt:

```bash
pip install quilt
quilt install story645/eas213
```


In [ ]:
# set the data and geo paths
datapath =
geopath = 

In [ ]:
from quilt.data.story645 import eas213
datapath = eas213.landsat_hires._data()
geopath = eas213.census._data()

In [ ]:
import geopandas as gpd
tracts = gpd.read_file(geopath, crs={'epsg':2263})

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10,10))

_ = tracts.plot(ax=ax, edgecolor='k', color='w')

In [ ]:
import xarray as xr
ds = xr.open_dataset('../data/landsat_hires.nc')

In [ ]:
import landsat_sample_functions as lf
# pull out the thermal radiances from from ds and convert them to temperature
tempK = lf.BT(ds['Thermal Radiances'], 10.9, 1)
#convert the temperature to Fahrenheit
tempF = 9/5*(tempK - 273) + 32

In [ ]:
fig, ax = plt.subplots()
_ = tempF.mean('time').plot(cmap='RdYlBu_r', ax=ax)
_ = ax.set_title(r"Temperature ($\degree F$")

# How do we combine the two?

In [ ]:
# let's stay w/ the mean to reduce the size of the data
temps = tempF.mean('time').to_dataframe().reset_index()

In [ ]:
temps.head()

In [ ]:
ds.lat.shape

In [ ]:
ds.dims

In [ ]:
temps['y'].max()

In [ ]:
#lets get back the lats and lons by indexing into dataset coordinates
lats = ds.lat.mean('time')[temps['y'].tolist()]
lons = ds.lon.mean('time')[temps['x'].tolist()]

In [ ]:
#convert to a geodataframe 
from shapely.geometry import Point
temps['Coordinates']  = list(zip(lats, lons))
temps['Coordinates'] = temps['Coordinates'].apply(Point)

In [ ]:
tempgdf = gpd.GeoDataFrame(temps, geometry='Coordinates', crs={'epsg':2263})

In [ ]:
tempgdf.head()

In [ ]:
tracts.head()

In [ ]:
# join the arrays such that all temps in a tract are assigned to that tract
tt = gpd.sjoin(tracts, tempgdf, how="inner", op='contains')

In [ ]:
tempgdf

In [ ]:
#look up your tract
# https://popfactfinder.planning.nyc.gov/profile/7547/census